In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import time
import matplotlib.pyplot as plt
from torchinfo import summary
import json
import os
import shutil
from torch.utils.data import random_split
import csv
from PIL import Image

In [53]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = ImageFolder("dane/train/", transform=train_transform)

dataset_size = len(full_dataset)
train_size = int(dataset_size * 0.92)
val_size = dataset_size - train_size
trainset, valset = random_split(full_dataset, [train_size, val_size])

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)
valloader = DataLoader(valset, batch_size=32, shuffle=False, num_workers=4)

print(f"Liczba klas: {len(full_dataset.classes)}")
print(f"Nazwy klas: {full_dataset.classes}")
print(f"Liczba obrazów treningowych: {len(trainset)}")
print(f"Liczba obrazów walidacyjnych: {len(valset)}")

Liczba klas: 50
Nazwy klas: ['acoustic', 'antenna', 'bacteria', 'battery', 'bean', 'beetle', 'bicycle', 'birch', 'bird', 'bomb', 'bread', 'bridge', 'camera', 'carbon', 'cat', 'corn', 'crab', 'crocodilian', 'echinoderm', 'egg', 'elephant', 'fish', 'flower', 'frog', 'fungus', 'gauge', 'hammer', 'icecream', 'kangaroo', 'memorial', 'monkey', 'motor', 'nest', 'palm', 'pizza', 'pot', 'printer', 'saw', 'snake', 'spice', 'spider', 'spoon', 'squash', 'swine', 'tea', 'tomato', 'towel', 'truck', 'turtle', 'worm']
Liczba obrazów treningowych: 80970
Liczba obrazów walidacyjnych: 7041


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = self.pool5(F.relu(self.bn5(self.conv5(x))))

        x = x.view(-1, 512 * 7 * 7)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Używane urządzenie: {device}")

Używane urządzenie: cuda:0


In [56]:
num_classes = len(full_dataset.classes)
model = CNNModel(num_classes).to(device)

model_path = 'model.pth'
if os.path.exists(model_path):
    print(f"Loading existing model from {model_path}")
    model.load_state_dict(torch.load(model_path))
else:
    print("No existing model found. Initializing new model.")

No existing model found. Initializing new model.


In [57]:
summary(model, input_size=(1, 3, 224, 224), verbose=2)

model_stats = summary(
    model,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [1, 50]                   --
├─Conv2d: 1-1                            [1, 32, 224, 224]         896
│    └─weight                                                      ├─864
│    └─bias                                                        └─32
├─BatchNorm2d: 1-2                       [1, 32, 224, 224]         64
│    └─weight                                                      ├─32
│    └─bias                                                        └─32
├─MaxPool2d: 1-3                         [1, 32, 74, 74]           --
├─Conv2d: 1-4                            [1, 64, 74, 74]           18,496
│    └─weight                                                      ├─18,432
│    └─bias                                                        └─64
├─BatchNorm2d: 1-5                       [1, 64, 74, 74]           128
│    └─weight                                                 

In [59]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)


def train_model(model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=10):
    os.makedirs("models", exist_ok=True)

    model.train()
    start_time = time.time()

    training_losses = []
    training_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        batch_count = len(trainloader)
        epoch_start = time.time()

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
            elapsed = time.time() - start_time
            epoch_elapsed = time.time() - epoch_start
            if (i + 1) % 50 == 0:
                print(f"\rEpoch {epoch+1}/{num_epochs} | Batch {i+1}/{batch_count} | Loss: {running_loss/(i+1):.4f} | Acc: {accuracy:.2f}% | Epoch time: {epoch_elapsed:.1f}s | Total time: {elapsed:.1f}s", end="")

        train_loss = running_loss / len(trainloader)
        train_acc = 100 * correct / total
        training_losses.append(train_loss)
        training_accuracies.append(train_acc)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for data in valloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        scheduler.step(val_loss)

        lr = optimizer.param_groups[0]['lr']
        print(f"\nEpoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, LR: {lr:.6f}")

        if epoch > 0:
            prev_backup = f"backup_model_epoch_{epoch}.pth"
            if os.path.exists(prev_backup):
                os.remove(prev_backup)

        backup_path = f"backup_model_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), backup_path)

        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            model_dir = f"models/model_{epoch+1}"
            os.makedirs(model_dir, exist_ok=True)

            model_path = f"{model_dir}/model.pth"
            torch.save(model.state_dict(), model_path)

            plt.figure(figsize=(12, 5))

            plt.subplot(1, 2, 1)
            plt.plot(training_losses, 'b-', label='Training')
            plt.plot(val_losses, 'r-', label='Validation')
            plt.title('Loss During Training')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()

            plt.subplot(1, 2, 2)
            plt.plot(training_accuracies, 'b-', label='Training')
            plt.plot(val_accuracies, 'r-', label='Validation')
            plt.title('Accuracy During Training')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy (%)')
            plt.legend()

            plt.tight_layout()
            plt.savefig(f"{model_dir}/training_history.png")
            plt.close()

            training_data = {
                'epoch': epoch + 1,
                'train_losses': training_losses,
                'train_accs': training_accuracies,
                'val_losses': val_losses,
                'val_accs': val_accuracies,
                'final_lr': lr
            }

            with open(f"{model_dir}/training_data.json", 'w') as f:
                json.dump(training_data, f)

    torch.save(model.state_dict(), 'main.pth')
    print("Training complete. Final model saved as main.pth")

    return {
        'train_losses': training_losses,
        'train_accs': training_accuracies,
        'val_losses': val_losses,
        'val_accs': val_accuracies
    }

history = train_model(model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=10)

/home/jszubzda/proj3-SSNE/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 | Batch 2500/2531 | Loss: 3.3359 | Acc: 12.62% | Epoch time: 118.4s | Total time: 118.4s
Epoch 1 - Train Loss: 3.3356, Train Acc: 12.62%, Val Loss: 3.0035, Val Acc: 20.98%, LR: 0.001000
Epoch 2/10 | Batch 2500/2531 | Loss: 3.1453 | Acc: 16.44% | Epoch time: 123.6s | Total time: 250.8s
Epoch 2 - Train Loss: 3.1452, Train Acc: 16.43%, Val Loss: 2.8189, Val Acc: 24.41%, LR: 0.001000
Epoch 3/10 | Batch 2500/2531 | Loss: 3.0018 | Acc: 19.82% | Epoch time: 125.8s | Total time: 388.1s
Epoch 3 - Train Loss: 3.0009, Train Acc: 19.82%, Val Loss: 2.6527, Val Acc: 29.71%, LR: 0.001000
Epoch 4/10 | Batch 2500/2531 | Loss: 2.8724 | Acc: 22.49% | Epoch time: 121.5s | Total time: 521.7s
Epoch 4 - Train Loss: 2.8733, Train Acc: 22.49%, Val Loss: 2.5282, Val Acc: 32.38%, LR: 0.001000
Epoch 5/10 | Batch 2500/2531 | Loss: 2.7883 | Acc: 24.90% | Epoch time: 121.9s | Total time: 655.5s
Epoch 5 - Train Loss: 2.7882, Train Acc: 24.91%, Val Loss: 2.4448, Val Acc: 34.04%, LR: 0.001000
Epoch 6/10 | Ba

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_losses'], 'b-', label='Training')
plt.plot(history['val_losses'], 'r-', label='Validation')
plt.title('Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_accs'], 'b-', label='Training')
plt.plot(history['val_accs'], 'r-', label='Validation')
plt.title('Accuracy During Training')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

# generate the test set predictions

In [ ]:
model.load_state_dict(torch.load('main.pth'))
model.eval()

test_dir = "dane/test/"
test_files = [f for f in os.listdir(test_dir)
              if f.lower().endswith(('.jpeg', '.jpg', '.png', '.JPEG'))]

print(f"Found {len(test_files)} test images")
print(f"Classes in training set: {full_dataset.classes}")

In [ ]:
predictions = []
filenames = []

with torch.no_grad():
    for file in test_files:
        img_path = os.path.join(test_dir, file)
        image = Image.open(img_path).convert('RGB')
        image_tensor = test_transform(image).unsqueeze(0).to(device)

        output = model(image_tensor)
        _, pred = torch.max(output, 1)

        base_name = os.path.splitext(file)[0]
        filename_jpeg = f"{base_name}.JPEG"

        predictions.append(pred.item())
        filenames.append(filename_jpeg)

        if len(predictions) % 100 == 0:
            print(f"Processed {len(predictions)}/{len(test_files)} images")

In [ ]:
with open('pred.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for filename, pred in zip(filenames, predictions):
        writer.writerow([filename, pred])